## Assignment 3 (Dependency Parser)

Everything should be done ON MY code, no new code.

1. Read https://aclanthology.org/D14-1082.pdf and maybe just write one paragraph summary in your README.md in your github

2. Do something called ablation study (meaning try to delete something so we know the impact of that deleted thing - very common in NLP)
- Recall that we have 18 word + 18 pos + 12 dep features
- Try to delete only the 12 dep features and check UAS
- Try to delete only the 18 pos features and check UAS
3. Do another comparison study testing the embedding
- Chaky uses some embedding
- Try to use (1) glove embedding (smallest), (2) nn.Embedding (train from scratch) and compare with Chaky's embedding - on how it affects the UAS
4. Do some testing, compare 2-3 sentences with spaCy and see whether our neural network gives the same dependency.

Criteria:
0: not done
1: ok
2: with comments/explanation like how Chaky does his tutorial

In [123]:
Name = "Md. Sakib Bin Alam"
ID = "st122574"

## Import libraries

In [1]:
# Import libraries
import sys
import numpy as np
import time
import os
import logging
from collections import Counter
from datetime import datetime
import math

from tqdm import tqdm  #gimmick for progressbar when you train
import pickle #saving and loading models

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import nn, optim

## 1. Parsing function

In [2]:
#basically, it takes the current state of the buffer, stack, dependencies
#tell us how SHIFT, LA, RA changes these three objects

class Parsing(object):
    
    #init stack, buffer, dep
    def __init__(self, sentence):  
        self.sentence = sentence     #['The', 'cat', 'sat]  #conll format which is already in the tokenized form
        self.stack    = ['ROOT']
        self.buffer   = sentence[:]  #in the beginning, everything is inside the buffer
        self.dep      = []           #maintains a list of tuples of dep
    
    #parse function that tells me how shift, la, ra changes these three objects
    def parse_step(self, transition):     #transition could be either S, LA, RA
        if transition == 'S':
            #get the top guy in the buffer and put in stack
            head = self.buffer.pop(0)
            self.stack.append(head)
        elif transition == 'LA':  #stack = [ROOT, He, has] ==> append to dep (has, he) and then He is gone from the stack [ROOT, has]
            dependent = self.stack.pop(-2)  #He
            self.dep.append((self.stack[-1], dependent))  #(has, he)
        elif transition == 'RA':
            #can you guys try to this???
            dependent = self.stack.pop()  #stack = [ROOT, has, control] ==> dep (has, control), control will be gone fromt he stack [ROOT, has]
            self.dep.append((self.stack[-1], dependent))
        else:
            print(f"Bad transition: {transition}")
    
    #given some series of transition, it gonna for-loop the parse function
    def parse(self, transitions):
        for t in transitions:
            self.parse_step(t)
        return self.dep
    
    #check whether things are finished - no need to do anymore functions....
    def is_completed(self):
        return (len(self.buffer) == 0) and (len(self.stack) == 1)  #so buffer is empty and ROOT is the only guy in stack

### Minibatch parsing

In [3]:
def minibatch_parse(sentences, model, batch_size):
    dep = []  #all the resulting dep
    
    #init Parsing instance for each sentence in the batch
    partial_parses = [Parsing(sentence) for sentence in sentences]  #in tokenized form
    #Parsing(['The', 'cat', 'sat']), Parsing(['Chaky', 'is', 'mad'])
    
    unfinished_parses = partial_parses[:]
    
    #while we still have sentence
    while unfinished_parses:  #if there are still a Parsing object
    
        #take a certain batch of sentence
        minibatch = unfinished_parses[:batch_size] #number of Parsing object
        
        #create a dummy model to tell us what's the next transition for each sentence
        transitions = model.predict(minibatch) 
        #transitions = [S, S, .....]
        #minibatch   = [Parsing(sentence1), Parsing(sentence2)]
        
                
        # for transition predicted this dummy model
        for transition, partial_parse in zip(transitions, minibatch):
            #parse step
            #transition: S
            #partial_parse: Parsing(sentence)
            partial_parse.parse_step(transition)
            
        #remove any sentence is finish
        unfinished_parses[:] = [p for p in unfinished_parses if not p.is_completed()]
    
    dep = [parse.dep for parse in partial_parses]
    
    return dep

## 2. Load data

In [4]:
def read_conll(filename):
    
    examples = []
    
    with open(filename) as f:
        i = 0
        word, pos, head, dep = [], [], [], []
        for line in f.readlines():
            i = i+1
            wa = line.strip().split('\t')  #['1', 'In', '_', 'ADP', 'IN', '_', '5', 'case', '_', '_']
            #In <--------  5th guy
            #     case
            
            if len(wa) == 10:  #if all the columns are there
                word.append(wa[1].lower())
                pos.append(wa[4])
                head.append(int(wa[6]))
                dep.append(wa[7])
            
            #the row is not exactly 10, it means new sentence
            elif len(word) > 0:  #if there is somethign inside the word
                examples.append({'word': word, 'pos': pos, 'head': head, 'dep': dep})  #in the sentence level
                word, pos, head, dep = [], [], [], [] #clear word, pos, head, dep
        
        if len(word) > 0:  #if there is somethign inside the word
            examples.append({'word': word, 'pos': pos, 'head': head, 'dep': dep})  #in the sentence level

    return examples        

In [5]:
def load_data():
    print("Loading data...")
    train_set = read_conll("train.conll")
    dev_set   = read_conll("dev.conll")
    test_set   = read_conll("test.conll")
    
    #make the dataset smaller so that machine can handle quickly
    train_set = train_set[:1000]
    dev_set   = dev_set[:500]
    test_set  = test_set[:500]
    
    return train_set, dev_set, test_set

# Part I: Ablation study
- Try to delete only the 12 dep features and check UAS
- Try to delete only the 18 pos features and check UAS

## 3. Parser

- create a tok2id dictionary in the __init__ function
- numercalize numercalize the words, dependencies, and pos tags
- create training data, create_instances by leveraging the ground truth of the dependencies
- finally the parse function

In [121]:
# modified Parser class to ensure dep features and pos features

P_PREFIX = '<p>:' #indicating pos tags
D_PREFIX = '<d>:' #indicating dependency tags
UNK      = '<UNK>'
NULL     = '<NULL>'
ROOT     = '<ROOT>'

class Parser(object):

    def __init__(self, dataset, pos_feat=True, dep_feat=True):
        
        self.n_features = 18
        self.pos_feat = pos_feat
        self.dep_feat = dep_feat

        tok2id = dict() 
        trans = ['L', 'R', 'S']
        self.n_trans = len(trans)
        self.n_deprel = 1
        
        # consider dep features
        if self.dep_feat == True:
            self.root_dep = 'root'
                
        #get all the dep of the dataset as list, e.g., ['root', 'acl', 'nmod', 'nmod:npmod']
         # Basically, if it's false we ill not create this list
            all_dep = [self.root_dep] + list(set([w for ex in dataset
                                                for w in ex['dep']
                                                if w != self.root_dep]))
        
        #1. put dep into tok2id lookup table, with D_PREFIX so we know it is dependency
        #{'D_PREFIX:root': 0, 'D_PREFIX:acl': 1, 'D_PREFIX:nmod': 2, ..., 'D_PREFIX:<NULL>': 30}
            tok2id = {D_PREFIX + l: i for (i, l) in enumerate(all_dep)}
            tok2id[D_PREFIX + NULL] = self.D_NULL = len(tok2id)
        
        #we are using "unlabeled" where we do not label with the dependency
        #thus the number of dependency relation is 1
            trans = ['L', 'R', 'S']
            self.n_deprel = 1   #because we are not predicting the relations, we are only predicting S, L, R
        
            #create a simple lookup table mapping action and id
            #e.g., tran2id: {'L': 0, 'R': 1, 'S': 2}
            #e.g., id2tran: {0: 'L', 1: 'R', 2: 'S'}
            self.n_trans = len(trans)
            self.tran2id = {t: i for (i, t) in enumerate(trans)}  #use for easy coding
            self.id2tran = {i: t for (i, t) in enumerate(trans)}

        # consider pos features
        if self.pos_feat == True:
            #2. put pos tags into tok2id lookup table, with P_PREFIX so we know it is pos
            tok2id.update(build_dict([P_PREFIX + w for ex in dataset for w in ex['pos']],
                                    offset=len(tok2id)))
            tok2id[P_PREFIX + UNK]  = self.P_UNK  = len(tok2id)  #also remember the pos tags of unknown
            tok2id[P_PREFIX + NULL] = self.P_NULL = len(tok2id)
            tok2id[P_PREFIX + ROOT] = self.P_ROOT = len(tok2id)
            
            #now tok2id:  {'P_PREFIX:root': 0, 'P_PREFIX:acl': 1, ..., 'P_PREFIX:JJR': 62, 'P_PREFIX:<UNK>': 63, 'P_PREFIX:<NULL>': 64, 'P_PREFIX:<ROOT>': 65}
        
        #3. put word into tok2id lookup table
        tok2id.update(build_dict([w for ex in dataset for w in ex['word']],
                                  offset=len(tok2id)))
        tok2id[UNK]  = self.UNK = len(tok2id)
        tok2id[NULL] = self.NULL = len(tok2id)
        tok2id[ROOT] = self.ROOT = len(tok2id)
        
        #now tok2id: {'D_PREFIX:root': 0, 'D_PREFIX:acl': 1, 'D_PREFIX:nmod': 2, ..., 'memory': 340, 'mr.': 341, '<UNK>': 342, '<NULL>': 343, '<ROOT>': 344}
        
        #create id2tok
        self.tok2id = tok2id
        self.id2tok = {v: k for (k, v) in tok2id.items()}
        
        # If some of the feature are not use, the number of features also change, so we need to fix this!
        if self.pos_feat == True:
            self.n_features = self.n_features + 18
        if self.dep_feat == True:
            self.n_features = self.n_features + 12
        
        self.n_tokens = len(tok2id)
        
    #utility function, in case we want to convert token to id
    #function to turn train set with words to train set with id instead using tok2id
    def numericalize(self, examples):
        numer_examples = []
        for ex in examples:
            word = [self.ROOT] + [self.tok2id[w] if w in self.tok2id
                                  else self.UNK for w in ex['word']]
            if self.pos_feat == True:
                pos  = [self.P_ROOT] + [self.tok2id[P_PREFIX + w] if P_PREFIX + w in self.tok2id
                                    else self.P_UNK for w in ex['pos']]
            head = [-1] + ex['head']
            if self.dep_feat == True:
                dep  = [-1] + [self.tok2id[D_PREFIX + w] if D_PREFIX + w in self.tok2id
                                else -1 for w in ex['dep']]
            
            if (self.pos_feat == False) and (self.dep_feat == False):
                numer_examples.append({'word': word, 'head': head})

            elif (self.pos_feat == False) and (self.dep_feat == True):
                numer_examples.append({'word': word, 'head': head, 'dep': dep})

            elif (self.pos_feat == True) and (self.dep_feat == False):
                numer_examples.append({'word': word, 'pos': pos,
                        'head': head})
            else:
                numer_examples.append({'word': word, 'pos': pos,
                                 'head': head, 'dep': dep})
        return numer_examples
    
    #function to extract features to form a feature embedding matrix
    def extract_features(self, stack, buf, arcs, ex):
             
        #ex['word']:  [55, 32, 33, 34, 35, 30], i.e., ['root', 'ms.', 'haag', 'plays', 'elianti', '.']
        #ex['pos']:   [29, 14, 14, 16, 14, 17], i.e., ['NNP', 'NNP', 'VBZ', 'NNP', '.']
        #ex['head']:  [-1, 2, 3, 0, 3, 3]  or ['root', 'compound', 'nsubj', 'root', 'dobj', 'punct']}
        #ex['dep']:   [-1, 1, 2, 0, 6, 12] or ['compound', 'nsubj', 'root', 'dobj', 'punct']

        #stack     :  [0]
        #buffer    :  [1, 2, 3, 4, 5]
        
        if stack[0] == "ROOT":
            stack[0] = 0  #start the stack with [ROOT]
        
        p_features = [] #pos features (2a, 2b, 2c) - 18
        d_features = [] #dep features (3b, 3c) - 12
        
        #last 3 things on the stack as features
        #if the stack is less than 3, then we simply append NULL from the left
        features = [self.NULL] * (3 - len(stack)) + [ex['word'][x] for x in stack[-3:]]
        
        # next 3 things on the buffer as features
        #if the buffer is less than 3, simply append NULL
        #the reason why NULL is appended on end because buffer is read left to right
        features += [ex['word'][x] for x in buf[:3]] + [self.NULL] * (3 - len(buf))
        
        if self.pos_feat == True:
            #corresponding pos tags
            p_features = [self.P_NULL] * (3 - len(stack)) + [ex['pos'][x] for x in stack[-3:]]
            p_features += [ex['pos'][x] for x in buf[:3]] + [self.P_NULL] * (3 - len(buf))
        
        #get leftmost children based on the dependency arcs
        def get_lc(k):
            return sorted([arc[1] for arc in arcs if arc[0] == k and arc[1] < k])

        #get right most children based on the dependency arcs
        def get_rc(k):
            return sorted([arc[1] for arc in arcs if arc[0] == k and arc[1] > k],
                          reverse=True)

        #get the leftmost and rightmost children of the top two words, thus we loop 2 times
        for i in range(2):
            if i < len(stack):
                k = stack[-i-1] #-1, -2 last two in the stack
                
                #the first and second lefmost/rightmost children of the top two words (i=1, 2) on the stack
                lc = get_lc(k)  
                rc = get_rc(k)
                
                #the leftmost of leftmost/rightmost of rightmost children of the top two words on the stack:
                llc = get_lc(lc[0]) if len(lc) > 0 else []
                rrc = get_rc(rc[0]) if len(rc) > 0 else []

                #(leftmost of first word on stack, rightmost of first word, 
                # leftmost of the second word on stack, rightmost of second, 
                # leftmost of leftmost, rightmost of rightmost
                features.append(ex['word'][lc[0]] if len(lc) > 0 else self.NULL)
                features.append(ex['word'][rc[0]] if len(rc) > 0 else self.NULL)
                features.append(ex['word'][lc[1]] if len(lc) > 1 else self.NULL)
                features.append(ex['word'][rc[1]] if len(rc) > 1 else self.NULL)
                features.append(ex['word'][llc[0]] if len(llc) > 0 else self.NULL)
                features.append(ex['word'][rrc[0]] if len(rrc) > 0 else self.NULL)

                if self.pos_feat == True:
                    #corresponding pos
                    p_features.append(ex['pos'][lc[0]] if len(lc) > 0 else self.P_NULL)
                    p_features.append(ex['pos'][rc[0]] if len(rc) > 0 else self.P_NULL)
                    p_features.append(ex['pos'][lc[1]] if len(lc) > 1 else self.P_NULL)
                    p_features.append(ex['pos'][rc[1]] if len(rc) > 1 else self.P_NULL)
                    p_features.append(ex['pos'][llc[0]] if len(llc) > 0 else self.P_NULL)
                    p_features.append(ex['pos'][rrc[0]] if len(rrc) > 0 else self.P_NULL)

                if self.dep_feat == True:
                    #corresponding dep
                    d_features.append(ex['dep'][lc[0]] if len(lc) > 0 else self.D_NULL)
                    d_features.append(ex['dep'][rc[0]] if len(rc) > 0 else self.D_NULL)
                    d_features.append(ex['dep'][lc[1]] if len(lc) > 1 else self.D_NULL)
                    d_features.append(ex['dep'][rc[1]] if len(rc) > 1 else self.D_NULL)
                    d_features.append(ex['dep'][llc[0]] if len(llc) > 0 else self.D_NULL)
                    d_features.append(ex['dep'][rrc[0]] if len(rrc) > 0 else self.D_NULL)
                
            else:
                #attach NULL when they don't exist
                features += [self.NULL] * 6
                if self.pos_feat == True:
                    p_features += [self.P_NULL] * 6
                if self.dep_feat == True:
                    d_features += [self.D_NULL] * 6

        if (self.pos_feat == True) and (self.dep_feat == True):
            features += p_features + d_features

        elif (self.pos_feat == False) and (self.dep_feat == True):
            features += d_features

        elif (self.pos_feat == True) and (self.dep_feat == False):
            features += p_features
        
        else:
            features = features

        assert len(features) == self.n_features  #assert they are 18 + 18 + 12
        
        return features

    #generate training examples
    #from the training sentences and their gold parse trees 
    def create_instances(self, examples):  #examples = word, pos, head, dep
        all_instances = []
        
        for i, ex in enumerate(examples):
            #Ms. Haag plays Elianti .
            #e.g., ex['word]: [344, 163, 99, 164, 165, 68]
            #here 344 stands for ROOT
            #Chaky - I cheated and take a look
            n_words = len(ex['word']) - 1  #excluding the root
            
            #arcs = {(head, tail, dependency label)}
            stack = [0]
            buf = [i + 1 for i in range(n_words)]  #[1, 2, 3, 4, 5]
            arcs = []
            instances = []
            
            #because that's the maximum number of shift, leftarcs, rightarcs you can have
            #this will determine the sample size of each training example
            #if given five words, we will get a sample of (10, 48) where 10 comes from 5 * 2, and 48 is n_features
            #but this for loop can be break if there is nothing left....
            
            for i in range(n_words * 2):  #maximum times you can do either S, L, R
                
                #get the gold transition based on the parse trees
                #gold_t can be either shift(2), leftarc(0), or rightarc(1)
                gold_t = self.get_oracle(stack, buf, ex)
                
                #if gold_t is None, no need to extract features.....
                if gold_t is None:
                    break
                
                #make sure when the model predicts, we inform the current state of stack and buffer, so
                #the model is not allowed to make any illegal action, e.g., buffer is empty but trying to pop
                legal_labels = self.legal_labels(stack, buf)                
                assert legal_labels[gold_t] == 1
                
                #extract the features that we want to use.
                features = self.extract_features(stack, buf, arcs, ex)
                instances.append((features, legal_labels, gold_t))
                
                #shift 
                if gold_t == 2:
                    stack.append(buf[0])
                    buf = buf[1:]
                #left arc 
                elif gold_t == 0:
                    arcs.append((stack[-1], stack[-2], gold_t))
                    stack = stack[:-2] + [stack[-1]]
                #right arc
                else:
                    arcs.append((stack[-2], stack[-1], gold_t - self.n_deprel))
                    stack = stack[:-1]
                    
            else:
                all_instances += instances

        if self.dep_feat == False: # We only need the word and pos
            all_instances = [[instance[0], instance[2]] for instance in all_instances]

        return all_instances
    
    #provide an one hot encoding of the labels
    def legal_labels(self, stack, buf):
        labels =  ([1] if len(stack) > 2  else [0]) * self.n_deprel  #left arc but you cannot do ROOT <--- He
        labels += ([1] if len(stack) >= 2 else [0]) * self.n_deprel  #right arc because ROOT --> He
        labels += [1] if len(buf) > 0 else [0]  #shift
        return labels
    
    #a simple function to check punctuation POS tags
    def punct(self, pos):
        return pos in ["''", ",", ".", ":", "``", "-LRB-", "-RRB-"]
    
    #decide whether to shift, leftarc, or rightarc, based on gold parse trees
    #this is needed to create training examples which contain samples and ground truth
    def get_oracle(self, stack, buf, ex):
        
        #leave if the stack is only 1, thus nothing to predict....
        if len(stack) < 2:
            return self.n_trans - 1
        
        #predict based on the last two words on the stack
        #stack: [ROOT, he, has]
        i0 = stack[-1] #has
        i1 = stack[-2] #he
        
        #get the head and dependency
        h0 = ex['head'][i0]
        h1 = ex['head'][i1]

        if self.dep_feat == True:
            d0 = ex['dep'][i0]
            d1 = ex['dep'][i1]
        
        #either shift, left arc or right arc
        #"Shift" = 2; "LA" = 0; "RA" = 1
        #if head of the second last word is the last word, then leftarc
        if (i1 > 0) and (h1 == i0):
            return 0  #action is left arc ---> gold_t
        #if head of the last word is the second last word, then rightarc
        #make sure nothing in the buffer has head with the last word on the stack
        #otherwise, we lose the last word.....
        elif (i1 >= 0) and (h0 == i1) and \
                (not any([x for x in buf if ex['head'][x] == i0])):
            return 1  #right arc
        #otherwise shift, if something is left in buffer, otherwise, do nothing....
        else:
            return None if len(buf) == 0 else 2  #shift
        
    def parse(self, dataset, eval_batch_size=5000):
        sentences = []
        sentence_id_to_idx = {}
        
        for i, example in enumerate(dataset):
            
            #example['word']=[188, 186, 186, ..., 59]
            #n_words=37
            #sentence=[1, 2, 3, 4, 5,.., 37]
            
            n_words = len(example['word']) - 1
            sentence = [j + 1 for j in range(n_words)]            
            sentences.append(sentence)
            
            #mapping the object unique id to the i            
            #The id is the object's memory address
            sentence_id_to_idx[id(sentence)] = i
            
        model = ModelWrapper(self, dataset, sentence_id_to_idx)
        dependencies = minibatch_parse(sentences, model, eval_batch_size)
        
        UAS = all_tokens = 0.0
        with tqdm(total=len(dataset)) as prog:
            for i, ex in enumerate(dataset):
                head = [-1] * len(ex['word'])
                for h, t, in dependencies[i]:
                    head[t] = h
                
                if (self.pos_feat == True) and (self.dep_feat == True):
                    for pred_h, gold_h, gold_l, pos in \
                            zip(head[1:], ex['head'][1:], ex['dep'][1:], ex['pos'][1:]):
                            assert self.id2tok[pos].startswith(P_PREFIX)
                            pos_str = self.id2tok[pos][len(P_PREFIX):]
                            if (not self.punct(pos_str)):
                                UAS += 1 if pred_h == gold_h else 0
                                all_tokens += 1

                elif (self.pos_feat == False) and (self.dep_feat == True):
                    for pred_h, gold_h, gold_l in \
                            zip(head[1:], ex['head'][1:], ex['dep'][1:]):
                            UAS += 1 if pred_h == gold_h else 0
                            all_tokens += 1

                elif (self.pos_feat == True) and (self.dep_feat == False):
                    for pred_h, gold_h, pos in \
                            zip(head[1:], ex['head'][1:], ex['pos'][1:]):
                            assert self.id2tok[pos].startswith(P_PREFIX)
                            pos_str = self.id2tok[pos][len(P_PREFIX):]
                            if (not self.punct(pos_str)):
                                UAS += 1 if pred_h == gold_h else 0
                                all_tokens += 1

                prog.update(i + 1)
        UAS /= all_tokens
        return UAS, dependencies

In [7]:
class ModelWrapper(object):
    def __init__(self, parser, dataset, sentence_id_to_idx):
        self.parser = parser
        self.dataset = dataset
        self.sentence_id_to_idx = sentence_id_to_idx

    def predict(self, partial_parses):
        mb_x = [self.parser.extract_features(p.stack, p.buffer, p.dep,
                                             self.dataset[self.sentence_id_to_idx[id(p.sentence)]])
                for p in partial_parses]
        mb_x = np.array(mb_x).astype('int32')
        mb_x = torch.from_numpy(mb_x).long()
        mb_l = [self.parser.legal_labels(p.stack, p.buffer) for p in partial_parses]

        pred = self.parser.model(mb_x)
        pred = pred.detach().numpy()
        
        #we need to multiply 10000 with legal labels, to force the model not to make any impossible prediction
        #other, when we parse sequentially, sometimes there is nothing in the buffer or stack, thus error....        
        pred = np.argmax(pred + 10000 * np.array(mb_l).astype('float32'), 1)
        pred = ["S" if p == 2 else ("LA" if p == 0 else "RA") for p in pred]
        
        return pred

In [8]:
#a simple function to create ids.....
def build_dict(keys, offset=0):
    #keys = ['P_PREFIX:IN', 'P_PREFIX:DT', 'P_PREFIX:NNP', 'P_PREFIX:CD', so on...]
    #offset is needed because this tok2id has something already inside....
    count = Counter()
    for key in keys:
        count[key] += 1
    
    #most_common = [('P_PREFIX:NN', 70), ('P_PREFIX:IN', 57), ... , ('P_PREFIX:JJR', 1)]
    #we use most_common in case we only want some maximum pos tags....
    mc = count.most_common()
    
    #{'P_PREFIX:NN': 31, 'P_PREFIX:IN': 32, .., 'P_PREFIX:JJR': 62} 
    return {w[0]: index + offset for (index, w) in enumerate(mc)}

## 4. Word Embedding

In [9]:
# print("4. Loading pretrained embeddings...",)
# start = time.time()
# word_vectors = {}
# for line in open("en-cw.txt").readlines():
#     we = line.strip().split() #we = word embeddings - first column: word;  the rest is embedding
#     word_vectors[we[0]] = [float(x) for x in we[1:]] #{word: [list of 50 numbers], nextword: [another list], so on...}
    
# #create an empty embedding matrix holding the embedding lookup table (vocab size, embed dim)
# #we use random.normal instead of zeros, to keep the embedding matrix arbitrary in case word vectors don't exist....
# embeddings_matrix = np.asarray(np.random.normal(0, 0.9, (parser.n_tokens, 50)), dtype='float32')

# for token in parser.tok2id:
#         i = parser.tok2id[token]
#         if token in word_vectors:
#             embeddings_matrix[i] = word_vectors[token]
#         elif token.lower() in word_vectors:
#             embeddings_matrix[i] = word_vectors[token.lower()]
# print("Embedding matrix shape (vocab, emb size): ", embeddings_matrix.shape)
# print("took {:.2f} seconds".format(time.time() - start))

## 5. Preprocessing

- Testing the parser
- Testing the numericalize
- Preprocessing (create instances)

In [10]:
# 1. word, pos_features, and dep_features

In [11]:
# Testing the parser

train_set, dev_set, test_set = load_data()

print("2. Building parser")
start = time.time()
parser = Parser(train_set)
print("took {:.2f} seconds".format(time.time() - start))

Loading data...
2. Building parser
took 0.03 seconds


In [12]:
# Testing the numericalize

#before numericalize
print("Word: ", train_set[1]["word"])
print("Pos: ",  train_set[1]["pos"])
print("Head: ", train_set[1]["head"])
print("Dep: ",  train_set[1]["dep"])

train_set = parser.numericalize(train_set)
dev_set   = parser.numericalize(dev_set)
test_set  = parser.numericalize(test_set)

#after numericalize
print("Word: ", train_set[1]["word"])
print("Pos: ",  train_set[1]["pos"])
print("Head: ", train_set[1]["head"])
print("Dep: ",  train_set[1]["dep"])

Word:  ['ms.', 'haag', 'plays', 'elianti', '.']
Pos:  ['NNP', 'NNP', 'VBZ', 'NNP', '.']
Head:  [2, 3, 0, 3, 3]
Dep:  ['compound', 'nsubj', 'root', 'dobj', 'punct']
Word:  [5156, 304, 1364, 1002, 2144, 87]
Pos:  [84, 42, 42, 55, 42, 46]
Head:  [-1, 2, 3, 0, 3, 3]
Dep:  [-1, 7, 11, 0, 10, 29]


In [13]:
# Preprocessing

print("5. Preprocessing training data...",)
start = time.time()
train_examples = parser.create_instances(train_set)
print("took {:.2f} seconds".format(time.time() - start))
print(train_examples[0])


5. Preprocessing training data...
took 1.70 seconds
([5155, 5155, 5156, 91, 113, 806, 5155, 5155, 5155, 5155, 5155, 5155, 5155, 5155, 5155, 5155, 5155, 5155, 83, 83, 84, 40, 41, 42, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38], [0, 0, 1], 2)


In [14]:
# word and dep_features (will not consider pos_features)

In [15]:
# Testing the parser

train_set, dev_set, test_set = load_data()

print("2. Building parser")
start = time.time()
parser = Parser(train_set, pos_t = False)
print("took {:.2f} seconds".format(time.time() - start))

Loading data...
2. Building parser
took 0.02 seconds


In [16]:
# Testing the numericalize

#before numericalize
print("Word: ", train_set[1]["word"])
print("Pos: ",  train_set[1]["pos"])
print("Head: ", train_set[1]["head"])
print("Dep: ",  train_set[1]["dep"])

train_set = parser.numericalize(train_set)
dev_set   = parser.numericalize(dev_set)
test_set  = parser.numericalize(test_set)

#after numericalize
print("Word: ", train_set[1]["word"])
try:
    print("Pos: ",  train_set[1]["pos"])
except:
    print('pos feature is not considered')
print("Head: ", train_set[1]["head"])
print("Dep: ",  train_set[1]["dep"])

Word:  ['ms.', 'haag', 'plays', 'elianti', '.']
Pos:  ['NNP', 'NNP', 'VBZ', 'NNP', '.']
Head:  [2, 3, 0, 3, 3]
Dep:  ['compound', 'nsubj', 'root', 'dobj', 'punct']
Word:  [5110, 258, 1318, 956, 2098, 41]
pos feature is not considered
Head:  [-1, 2, 3, 0, 3, 3]
Dep:  [-1, 7, 11, 0, 10, 29]


In [17]:
# Preprocessing

print("5. Preprocessing training data...",)
start = time.time()
train_examples = parser.create_instances(train_set)
print("took {:.2f} seconds".format(time.time() - start))
print(train_examples[0])

5. Preprocessing training data...
took 1.43 seconds
([5109, 5109, 5110, 45, 67, 760, 5109, 5109, 5109, 5109, 5109, 5109, 5109, 5109, 5109, 5109, 5109, 5109, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38], [0, 0, 1], 2)


In [ ]:
# dep_t = False

train_set, dev_set, test_set = load_data()

print("2. Building parser")
start = time.time()
parser = Parser(train_set, dep_t=False)
print("took {:.2f} seconds".format(time.time() - start))

#before numericalize
print("Word: ", train_set[1]["word"])
print("Pos: ",  train_set[1]["pos"])
print("Head: ", train_set[1]["head"])
print("Dep: ",  train_set[1]["dep"])

train_set = parser.numericalize(train_set)
dev_set   = parser.numericalize(dev_set)
test_set  = parser.numericalize(test_set)

#after numericalize
print("Word: ", train_set[1]["word"])
print("Pos: ",  train_set[1]["pos"])
print("Head: ", train_set[1]["head"])
try:
    print("Dep: ",  train_set[1]["dep"])
except:
    print("DEP feature False")

print("5. Preprocessing training data...",)
start = time.time()
train_examples = parser.create_instances(train_set)
print("took {:.2f} seconds".format(time.time() - start))
print(train_examples[0])

Loading data...
2. Building parser
took 0.02 seconds
Word:  ['ms.', 'haag', 'plays', 'elianti', '.']
Pos:  ['NNP', 'NNP', 'VBZ', 'NNP', '.']
Head:  [2, 3, 0, 3, 3]
Dep:  ['compound', 'nsubj', 'root', 'dobj', 'punct']
Word:  [5117, 265, 1325, 963, 2105, 48]
Pos:  [45, 3, 3, 16, 3, 7]
Head:  [-1, 2, 3, 0, 3, 3]
DEP feature False
5. Preprocessing training data...
took 1.60 seconds
[[5116, 5116, 5117, 52, 74, 767, 5116, 5116, 5116, 5116, 5116, 5116, 5116, 5116, 5116, 5116, 5116, 5116, 44, 44, 45, 1, 2, 3, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44], 2]


## 6. Minibatch Loader

In [19]:
def get_minibatches(data, minibatch_size, shuffle=True):
    data_size = len(data[0])
    indices = np.arange(data_size)
    if shuffle:
        np.random.shuffle(indices)
    for minibatch_start in np.arange(0, data_size, minibatch_size):
        minibatch_indices = indices[minibatch_start:minibatch_start + minibatch_size]
        yield [_minibatch(d, minibatch_indices) for d in data]

def _minibatch(data, minibatch_idx):
    return data[minibatch_idx] if type(data) is np.ndarray else [data[i] for i in minibatch_idx]

# def minibatches(data, batch_size):
#     x = np.array([d[0] for d in data])
#     y = np.array([d[2] for d in data])
#     one_hot = np.zeros((y.size, 3))
#     one_hot[np.arange(y.size), y] = 1
#     return get_minibatches([x, one_hot], batch_size)

# update the minibatches function so that it works without dep features

def minibatches_update(data, batch_size, dep_t=True):

    if dep_t == True:
        x = np.array([d[0] for d in data])
        y = np.array([d[2] for d in data])
        #print(x.shape)
        #print(y.shape)
        one_hot = np.zeros((y.size, 3))
        #print(one_hot.shape)
        one_hot[np.arange(y.size), y] = 1
    
    if dep_t == False:
        x = np.array([d[0] for d in data])
        y = np.array([d[1] for d in data])
        #print(x.shape)
        #print(y.shape)
        one_hot = np.zeros((y.size, 3))
        #print(one_hot.shape)
        one_hot[np.arange(y.size), y] = 1

    return get_minibatches([x, one_hot], batch_size)

## 7. Neural Network

In [20]:
class ParserModel(nn.Module):

    def __init__(self, embeddings, n_features=48,
                 hidden_size=400, n_classes=3, dropout_prob=0.5):

        super(ParserModel, self).__init__()
        self.n_features   = n_features
        self.n_classes    = n_classes
        self.dropout_prob = dropout_prob
        self.embed_size   = embeddings.shape[1]
        self.hidden_size  = hidden_size
        self.pretrained_embeddings = nn.Embedding(embeddings.shape[0], self.embed_size)
        self.pretrained_embeddings.weight = nn.Parameter(torch.tensor(embeddings))

        self.embed_to_hidden = nn.Linear(n_features * self.embed_size, hidden_size)
        self.dropout = nn.Dropout(p=dropout_prob)
        self.hidden_to_logits = nn.Linear(hidden_size, n_classes)

    def embedding_lookup(self, t):
        #t:  batch_size, n_features
        batch_size = t.size()[0]
                    
        x = self.pretrained_embeddings(t)        
        x = x.reshape(-1, self.n_features * self.embed_size)
        # x = (1024, 48 * 50)

        return x

    def forward(self, t):
        # t: (1024, 48)
        embeddings = self.embedding_lookup(t)  
    
        # embeddings: (1024, 48 * 50)
        hidden = self.embed_to_hidden(embeddings)
    
        # hidden: (1024, 200)
        hidden_activations = F.relu(hidden)
        # hidden_activations: (1024, 200)
        thin_net = self.dropout(hidden_activations)
        # thin_net: (1024, 200)
        logits = self.hidden_to_logits(thin_net)
        # logits: (1024, 3)

        return logits

In [21]:
#just a class to get the average.....
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [22]:
def train(parser, train_data, dev_data, output_path, batch_size=1024, n_epochs=10, lr=0.0005, dep_t=True):
    
    best_dev_UAS = 0
    
    optimizer = optim.Adam(parser.model.parameters(), lr=0.001)
    loss_func = nn.CrossEntropyLoss()

    for epoch in range(n_epochs):
        print("Epoch {:} out of {:}".format(epoch + 1, n_epochs))
        dev_UAS = train_for_epoch(
            parser, train_data, dev_data, optimizer, loss_func, batch_size, dep_t)
        print(f'Current UAS scores = {dev_UAS}')
        if dev_UAS > best_dev_UAS:
            best_dev_UAS = dev_UAS
            print("New best dev UAS! Saving model.")
            torch.save(parser.model.state_dict(), output_path)
        print("")


def train_for_epoch(parser, train_data, dev_data, optimizer, loss_func, batch_size, dep_t=True):
    
    parser.model.train()  # Places model in "train" mode, i.e. apply dropout layer
    n_minibatches = math.ceil(len(train_data) / batch_size)
    loss_meter = AverageMeter()

    with tqdm(total=(n_minibatches)) as prog:
        if dep_t == True:
            for i, (train_x, train_y) in enumerate(minibatches_update(train_data, batch_size)):
                
                #train_x:  batch_size, n_features
                #train_y:  batch_size, target(=3)
                
                optimizer.zero_grad() 
                loss = 0.
                train_x = torch.from_numpy(train_x).long()  #long() for int so embedding works....
                train_y = torch.from_numpy(train_y.nonzero()[1]).long()  #get the index with 1 because torch expects label to be single integer

                # Forward pass: compute predicted logits.
                logits = parser.model(train_x)
                # Compute loss
                #print(logits.shape, train_y.shape)
                loss = loss_func(logits, train_y)
                # Compute gradients of the loss w.r.t model parameters.
                loss.backward()
                # Take step with optimizer.
                optimizer.step()

                prog.update(1)
                loss_meter.update(loss.item())
        else:
            for i, (train_x, train_y) in enumerate(minibatches_update(train_data, batch_size, dep_t=False)):
            
                #train_x:  batch_size, n_features
                #train_y:  batch_size, target(=3)
                
                optimizer.zero_grad() 
                loss = 0.
                train_x = torch.from_numpy(train_x).long()  #long() for int so embedding works....
                train_y = torch.from_numpy(train_y.nonzero()[1]).long()  #get the index with 1 because torch expects label to be single integer

                # Forward pass: compute predicted logits.
                logits = parser.model(train_x)
                # Compute loss
                #print(logits.shape, train_y.shape)
                loss = loss_func(logits, train_y)
                # Compute gradients of the loss w.r.t model parameters.
                loss.backward()
                # Take step with optimizer.
                optimizer.step()

                prog.update(1)
                loss_meter.update(loss.item())

    print("Average Train Loss: {}".format(loss_meter.avg))
    print("Evaluating on dev set",)
    parser.model.eval()  # Places model in "eval" mode, i.e. don't apply dropout layer
        
    dev_UAS, _ = parser.parse(dev_data)
    print("- dev UAS: {:.2f}".format(dev_UAS * 100.0))
    return dev_UAS

## 8. Training and Testing

### Training with all features

In [30]:
train_set, dev_set, test_set = load_data()

print("Building parser")
start = time.time()
parser = Parser(train_set)
print("took {:.2f} seconds".format(time.time() - start))

train_set = parser.numericalize(train_set)
dev_set   = parser.numericalize(dev_set)
test_set  = parser.numericalize(test_set)

Loading data...
Building parser
took 0.03 seconds


In [31]:
print("4. Loading pretrained embeddings...",)
start = time.time()
word_vectors = {}
for line in open("en-cw.txt").readlines():
    we = line.strip().split() #we = word embeddings - first column: word;  the rest is embedding
    word_vectors[we[0]] = [float(x) for x in we[1:]] #{word: [list of 50 numbers], nextword: [another list], so on...}
    
#create an empty embedding matrix holding the embedding lookup table (vocab size, embed dim)
#we use random.normal instead of zeros, to keep the embedding matrix arbitrary in case word vectors don't exist....
embeddings_matrix = np.asarray(np.random.normal(0, 0.9, (parser.n_tokens, 50)), dtype='float32')

for token in parser.tok2id:
        i = parser.tok2id[token]
        if token in word_vectors:
            embeddings_matrix[i] = word_vectors[token]
        elif token.lower() in word_vectors:
            embeddings_matrix[i] = word_vectors[token.lower()]
print("Embedding matrix shape (vocab, emb size): ", embeddings_matrix.shape)
print("took {:.2f} seconds".format(time.time() - start))

4. Loading pretrained embeddings...
Embedding matrix shape (vocab, emb size):  (5157, 50)
took 2.78 seconds


In [32]:
print("Preprocessing training data...",)
start = time.time()
train_examples = parser.create_instances(train_set)
print("took {:.2f} seconds".format(time.time() - start))

Preprocessing training data...
took 1.93 seconds


In [33]:
#create directory if it does not exist for saving the weights...
output_dir = "output/{:%Y%m%d_%H%M%S}/".format(datetime.now())
output_path = output_dir + "model.weights_all"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
print(80 * "=")
print("TRAINING")
print(80 * "=")
    
model = ParserModel(embeddings_matrix)
parser.model = model

start = time.time()
train(parser, train_examples, dev_set, output_path,
      batch_size=1024, n_epochs=10, lr=0.0005)

TRAINING
Epoch 1 out of 10


100%|██████████| 48/48 [00:01<00:00, 25.95it/s]


Average Train Loss: 0.5864982747783264
Evaluating on dev set


125250it [00:00, 4743316.89it/s]       


- dev UAS: 57.82
Current UAS scores = 0.5781842304776346
New best dev UAS! Saving model.

Epoch 2 out of 10


...

- dev UAS: 73.82
Current UAS scores = 0.7381539044730857
New best dev UAS! Saving model.

Epoch 10 out of 10


100%|██████████| 48/48 [00:01<00:00, 24.02it/s]


Average Train Loss: 0.11471605145682891
Evaluating on dev set


125250it [00:00, 6033358.32it/s]       

- dev UAS: 74.11
Current UAS scores = 0.7410917361637605
New best dev UAS! Saving model.



### Testing

In [34]:
print(80 * "=")
print("Testing [pos_feature False]...")
print(80 * "=")

print("Restoring the best model weights found on the dev set")
parser.model.load_state_dict(torch.load(output_path))
print("Final evaluation on test set",)
parser.model.eval()
UAS, dependencies = parser.parse(test_set)
print("- test UAS: {:.2f}".format(UAS * 100.0))
print("Done!")

Testing [pos_feature False]...
Restoring the best model weights found on the dev set
Final evaluation on test set


125250it [00:00, 6567117.64it/s]       

- test UAS: 73.93
Done!


### Training with dep features (pos_feature = False)

In [23]:
train_set, dev_set, test_set = load_data()

print("Building parser")
start = time.time()
parser = Parser(train_set, pos_t=False)
print("took {:.2f} seconds".format(time.time() - start))

train_set = parser.numericalize(train_set)
dev_set   = parser.numericalize(dev_set)
test_set  = parser.numericalize(test_set)

Loading data...
Building parser
took 0.02 seconds


In [24]:
print("4. Loading pretrained embeddings...",)
start = time.time()
word_vectors = {}
for line in open("en-cw.txt").readlines():
    we = line.strip().split() #we = word embeddings - first column: word;  the rest is embedding
    word_vectors[we[0]] = [float(x) for x in we[1:]] #{word: [list of 50 numbers], nextword: [another list], so on...}
    
#create an empty embedding matrix holding the embedding lookup table (vocab size, embed dim)
#we use random.normal instead of zeros, to keep the embedding matrix arbitrary in case word vectors don't exist....
embeddings_matrix = np.asarray(np.random.normal(0, 0.9, (parser.n_tokens, 50)), dtype='float32')

for token in parser.tok2id:
        i = parser.tok2id[token]
        if token in word_vectors:
            embeddings_matrix[i] = word_vectors[token]
        elif token.lower() in word_vectors:
            embeddings_matrix[i] = word_vectors[token.lower()]
print("Embedding matrix shape (vocab, emb size): ", embeddings_matrix.shape)
print("took {:.2f} seconds".format(time.time() - start))

4. Loading pretrained embeddings...
Embedding matrix shape (vocab, emb size):  (5111, 50)
took 2.54 seconds


In [25]:
print("Preprocessing training data...",)
start = time.time()
train_examples = parser.create_instances(train_set)
print("took {:.2f} seconds".format(time.time() - start))

Preprocessing training data...
took 1.54 seconds


In [26]:
#create directory if it does not exist for saving the weights...
output_dir = "output/{:%Y%m%d_%H%M%S}/".format(datetime.now())
output_path = output_dir + "model.weights_dep"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
print(80 * "=")
print("TRAINING")
print(80 * "=")
    
model = ParserModel(embeddings_matrix, n_features=30) # since pos_feature = False
parser.model = model

start = time.time()
train(parser, train_examples, dev_set, output_path,
      batch_size=1024, n_epochs=10, lr=0.0005)

TRAINING
Epoch 1 out of 10


100%|██████████| 48/48 [00:01<00:00, 37.53it/s]


Average Train Loss: 0.574447072421511
Evaluating on dev set


125250it [00:00, 18011333.90it/s]      


- dev UAS: 51.40
Current UAS scores = 0.5140381571273848
New best dev UAS! Saving model.

Epoch 2 out of 10


...

- dev UAS: 69.03
Current UAS scores = 0.6903274181454636
New best dev UAS! Saving model.

Epoch 10 out of 10


100%|██████████| 48/48 [00:01<00:00, 36.55it/s]


Average Train Loss: 0.11791128277157743
Evaluating on dev set


125250it [00:00, 16815613.33it/s]      

- dev UAS: 68.39
Current UAS scores = 0.6839123552445222



### Testing with Dep features

In [27]:
print(80 * "=")
print("Testing [pos_feature False]...")
print(80 * "=")

print("Restoring the best model weights found on the dev set")
parser.model.load_state_dict(torch.load(output_path))
print("Final evaluation on test set",)
parser.model.eval()
UAS, dependencies = parser.parse(test_set)
print("- test UAS: {:.2f}".format(UAS * 100.0))
print("Done!")

Testing [pos_feature False]...
Restoring the best model weights found on the dev set
Final evaluation on test set


125250it [00:00, 15469274.91it/s]      

- test UAS: 71.17
Done!


### Training with pos features (dep_feature = False)

In [35]:
train_set, dev_set, test_set = load_data()

print("Building parser")
start = time.time()
parser = Parser(train_set, dep_t=False)
print("took {:.2f} seconds".format(time.time() - start))

train_set = parser.numericalize(train_set)
dev_set   = parser.numericalize(dev_set)
test_set  = parser.numericalize(test_set)

print("4. Loading pretrained embeddings...",)
start = time.time()
word_vectors = {}
for line in open("en-cw.txt").readlines():
    we = line.strip().split() #we = word embeddings - first column: word;  the rest is embedding
    word_vectors[we[0]] = [float(x) for x in we[1:]] #{word: [list of 50 numbers], nextword: [another list], so on...}
    
#create an empty embedding matrix holding the embedding lookup table (vocab size, embed dim)
#we use random.normal instead of zeros, to keep the embedding matrix arbitrary in case word vectors don't exist....
embeddings_matrix = np.asarray(np.random.normal(0, 0.9, (parser.n_tokens, 50)), dtype='float32')

for token in parser.tok2id:
        i = parser.tok2id[token]
        if token in word_vectors:
            embeddings_matrix[i] = word_vectors[token]
        elif token.lower() in word_vectors:
            embeddings_matrix[i] = word_vectors[token.lower()]
print("Embedding matrix shape (vocab, emb size): ", embeddings_matrix.shape)
print("took {:.2f} seconds".format(time.time() - start))

print("5. Preprocessing training data...",)
start = time.time()
train_examples = parser.create_instances(train_set)
print("took {:.2f} seconds".format(time.time() - start))

Loading data...
Building parser
took 0.03 seconds
4. Loading pretrained embeddings...
Embedding matrix shape (vocab, emb size):  (5118, 50)
took 2.72 seconds
5. Preprocessing training data...
took 1.71 seconds


In [36]:
#create directory if it does not exist for saving the weights...
output_dir = "output_pos/{:%Y%m%d_%H%M%S}".format(datetime.now())
output_path = output_dir + "model.weights_pos"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
print(80 * "=")
print("TRAINING")
print(80 * "=")
    
model = ParserModel(embeddings_matrix, n_features=36) # Because now we only use 2 features (18 + 18)
parser.model = model

start = time.time()
train(parser, train_examples, dev_set, output_path,
      batch_size=1024, n_epochs=10, lr=0.0005, dep_t=False)

TRAINING
Epoch 1 out of 10


100%|██████████| 48/48 [00:01<00:00, 31.18it/s]


Average Train Loss: 0.4769222792237997
Evaluating on dev set


125250it [00:00, 6426921.65it/s]       


- dev UAS: 63.91
Current UAS scores = 0.6391205458680819
New best dev UAS! Saving model.

Epoch 2 out of 10


...

- dev UAS: 77.81
Current UAS scores = 0.7781463229719484
New best dev UAS! Saving model.

Epoch 10 out of 10


100%|██████████| 48/48 [00:01<00:00, 33.08it/s]


Average Train Loss: 0.07978511873322229
Evaluating on dev set


125250it [00:00, 6717171.85it/s]       

- dev UAS: 78.13
Current UAS scores = 0.7812736921910538
New best dev UAS! Saving model.



### Testing with POS features

In [37]:
print(80 * "=")
print("Testing [pos_feature False]...")
print(80 * "=")

print("Restoring the best model weights found on the dev set")
parser.model.load_state_dict(torch.load(output_path))
print("Final evaluation on test set",)
parser.model.eval()
UAS, dependencies = parser.parse(test_set)
print("- test UAS: {:.2f}".format(UAS * 100.0))
print("Done!")

Testing [pos_feature False]...
Restoring the best model weights found on the dev set
Final evaluation on test set


125250it [00:00, 6847274.26it/s]       

- test UAS: 78.93
Done!


# Observation

| Features         | Dev UAS | Test UAS |
|----------------: |---------|:--------:|
|  Words, POS, DEP | 73.91   | 74.11    |
|  Words, DEP      | 69.13   | 71.17    |
|  Words, POS      | 78.13   | 78.93    |

So, we find that words + pos is giving the best score. Therefore, pos is very significant features.

# Part II: Embedding Comparison
- Chaky uses some embedding
- Try to use (1) glove embedding (smallest), (2) nn.Embedding (train from scratch) and compare with Chaky's embedding - on how it affects the UAS

***I will select dep_feature = False since it's giving the best result.***

## Glove (pre-trained model)

## Training with Pos features

In [81]:
train_set, dev_set, test_set = load_data()

print("Building parser")
start = time.time()
parser = Parser(train_set, dep_t=False)
print("took {:.2f} seconds".format(time.time() - start))

train_set = parser.numericalize(train_set)
dev_set   = parser.numericalize(dev_set)
test_set  = parser.numericalize(test_set)

print("Loading pretrained Glove embeddings...")
start = time.time()
word_vectors = {}
for line in open("glove.6B.50d.txt", encoding="utf8").readlines():
    we = line.strip().split() #we = word embeddings - first column: word;  the rest is embedding
    word_vectors[we[0]] = [float(x) for x in we[1:]] #{word: [list of 50 numbers], nextword: [another list], so on...}
    
#create an empty embedding matrix holding the embedding lookup table (vocab size, embed dim)
#we use random.normal instead of zeros, to keep the embedding matrix arbitrary in case word vectors don't exist....
embeddings_matrix = np.asarray(np.random.normal(0, 0.9, (parser.n_tokens, 50)), dtype='float32')

for token in parser.tok2id:
        i = parser.tok2id[token]
        if token in word_vectors:
            embeddings_matrix[i] = word_vectors[token]
        elif token.lower() in word_vectors:
            embeddings_matrix[i] = word_vectors[token.lower()]
print("Embedding matrix shape (vocab, emb size): ", embeddings_matrix.shape)
print("took {:.2f} seconds".format(time.time() - start))

print("5. Preprocessing training data...",)
start = time.time()
train_examples = parser.create_instances(train_set)
print("took {:.2f} seconds".format(time.time() - start))

Loading data...
Building parser
took 0.03 seconds
Loading pretrained Glove embeddings...
Embedding matrix shape (vocab, emb size):  (5118, 50)
took 8.06 seconds
5. Preprocessing training data...
took 1.94 seconds


In [82]:
#create directory if it does not exist for saving the weights...
output_dir = "output_pos/{:%Y%m%d_%H%M%S}".format(datetime.now())
output_path = output_dir + "glove_weights"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
print(80 * "=")
print("TRAINING")
print(80 * "=")
    
model = ParserModel(embeddings_matrix, n_features=36) # Because now we only use 2 features (18 + 18)
parser.model = model

start = time.time()
train(parser, train_examples, dev_set, output_path,
      batch_size=1024, n_epochs=10, lr=0.0005, dep_t=False)

TRAINING
Epoch 1 out of 10


100%|██████████| 48/48 [00:01<00:00, 32.23it/s]


Average Train Loss: 0.5826965409020582
Evaluating on dev set


125250it [00:00, 6753960.76it/s]       


- dev UAS: 52.63
Current UAS scores = 0.5262509476876421
New best dev UAS! Saving model.

Epoch 2 out of 10


...

- dev UAS: 76.18
Current UAS scores = 0.7618460955269143
New best dev UAS! Saving model.

Epoch 10 out of 10


100%|██████████| 48/48 [00:01<00:00, 31.43it/s]


Average Train Loss: 0.10859201460455854
Evaluating on dev set


125250it [00:00, 4787756.45it/s]       

- dev UAS: 76.04
Current UAS scores = 0.7604245640636846



### Testing with Pos features

In [83]:
print(80 * "=")
print("Testing [pos_feature False]...")
print(80 * "=")

print("Restoring the best model weights found on the dev set")
parser.model.load_state_dict(torch.load(output_path))
print("Final evaluation on test set",)
parser.model.eval()
UAS, dependencies = parser.parse(test_set)
print("- test UAS: {:.2f}".format(UAS * 100.0))
print("Done!")

Testing [pos_feature False]...
Restoring the best model weights found on the dev set
Final evaluation on test set


125250it [00:00, 6212075.35it/s]       

- test UAS: 77.45
Done!


## Embedding train from scratch

In [84]:
class ParserModel2(nn.Module):

    def __init__(self, embeddings, n_features=48,
                 hidden_size=400, n_classes=3, dropout_prob=0.5):

        super(ParserModel2, self).__init__()
        self.n_features   = n_features
        self.n_classes    = n_classes
        self.dropout_prob = dropout_prob
        self.embed_size   = embeddings.shape[1]
        self.hidden_size  = hidden_size
        self.embeddings = nn.Embedding(embeddings.shape[0], self.embed_size) # Modified
        
        self.embed_to_hidden = nn.Linear(n_features * self.embed_size, hidden_size)
        self.dropout = nn.Dropout(p=dropout_prob)
        self.hidden_to_logits = nn.Linear(hidden_size, n_classes)

    def embedding_lookup(self, t):
        #t:  batch_size, n_features
        batch_size = t.size()[0]
                    
        x = self.embeddings(t) # Modified
        x = x.reshape(-1, self.n_features * self.embed_size)
        # x = (1024, 48 * 50)

        return x

    def forward(self, t):
        # t: (1024, 48)
        embeddings = self.embedding_lookup(t)  
    
        # embeddings: (1024, 48 * 50)
        hidden = self.embed_to_hidden(embeddings)
    
        # hidden: (1024, 200)
        hidden_activations = F.relu(hidden)
        # hidden_activations: (1024, 200)
        thin_net = self.dropout(hidden_activations)
        # thin_net: (1024, 200)
        logits = self.hidden_to_logits(thin_net)
        # logits: (1024, 3)

        return logits

In [86]:
#create directory if it does not exist for saving the weights...
output_dir = "output_pos/{:%Y%m%d_%H%M%S}".format(datetime.now())
output_path = output_dir + "embedding_scractch_weights"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
print(80 * "=")
print("TRAINING")
print(80 * "=")
    
model = ParserModel2(embeddings_matrix, n_features=36) # Because now we only use 2 features (18 + 18)
parser.model = model

start = time.time()
train(parser, train_examples, dev_set, output_path,
      batch_size=1024, n_epochs=10, lr=0.0005, dep_t=False)

TRAINING
Epoch 1 out of 10


100%|██████████| 48/48 [00:01<00:00, 33.15it/s]


Average Train Loss: 0.5773063705613216
Evaluating on dev set


125250it [00:00, 6833379.85it/s]       


- dev UAS: 52.93
Current UAS scores = 0.5292835481425322
New best dev UAS! Saving model.

Epoch 2 out of 10


...

- dev UAS: 74.47
Current UAS scores = 0.7446929492039424
New best dev UAS! Saving model.

Epoch 10 out of 10


100%|██████████| 48/48 [00:01<00:00, 33.97it/s]


Average Train Loss: 0.11315890510256092
Evaluating on dev set


125250it [00:00, 6759174.70it/s]       

- dev UAS: 73.77
Current UAS scores = 0.7376800606520091



## Testing

In [87]:
print(80 * "=")
print("Testing [pos_feature False]...")
print(80 * "=")

print("Restoring the best model weights found on the dev set")
parser.model.load_state_dict(torch.load(output_path))
print("Final evaluation on test set",)
parser.model.eval()
UAS, dependencies = parser.parse(test_set)
print("- test UAS: {:.2f}".format(UAS * 100.0))
print("Done!")

Testing [pos_feature False]...
Restoring the best model weights found on the dev set
Final evaluation on test set


125250it [00:00, 6774340.74it/s]       

- test UAS: 74.32
Done!


# Observation

| Embedding case   | Dev UAS | Test UAS |
|----------------: |---------|:--------:|
|  Glove           | 76.18   | 77.45    |
|  Embedding scratch| 74.47  | 74.12    |

So, we see that Glove embedding is giving the better result. Interestingly embedding from scratch also giving a very good result. Ultimately by comparing all in (words + pos features) is giving the best result, that was done in previous analysis.



# Part IV: 4. Do some testing, compare 2-3 sentences with spaCy and see whether our neural network gives the same dependency.

In [88]:
_, _, test_set = load_data()

Loading data...


In [89]:
test_sentences = []


for sent in test_set:
    if len(sent['word']) <= 10:
        test_sentences.append(sent)

In [90]:
len(test_sentences)

46

In [94]:
three_test_sentences = test_sentences[:3]
len(three_test_sentence)

3

In [95]:
#three_test_sentence
final_sentences = []

for sent in three_test_sentences:
    sentence_join = ' '.join(sent['word'])
    final_sentences.append(sentence_join)

In [96]:
final_sentences

["no , it was n't black monday .",
 'the finger-pointing has already begun .',
 '`` the equity market was illiquid .']

## Spacy

In [97]:
import spacy
from spacy import displacy #displacy is for visualization
nlp = spacy.load("en_core_web_sm")

for sent in final_sentences:
    doc = nlp(sent)
    options = {"collapse_punct": False}
    displacy.render(doc, options = options, style="dep", jupyter=True)

## Our Model

In [108]:
model = ParserModel2(embeddings_matrix, n_features=36)
parser.model = model

start = time.time()
train(parser, train_examples, dev_set, output_path,
      batch_size=1024, n_epochs=10, lr=0.0005, dep_t=False)

Epoch 1 out of 10


100%|██████████| 48/48 [00:01<00:00, 32.66it/s]


Average Train Loss: 0.6023708507418633
Evaluating on dev set


125250it [00:00, 6792296.34it/s]       


- dev UAS: 55.66
Current UAS scores = 0.5565769522365428
New best dev UAS! Saving model.

Epoch 2 out of 10


...

- dev UAS: 74.37
Current UAS scores = 0.7436504927975739
New best dev UAS! Saving model.

Epoch 10 out of 10


100%|██████████| 48/48 [00:01<00:00, 33.99it/s]


Average Train Loss: 0.12194157112389803
Evaluating on dev set


125250it [00:00, 6149969.87it/s]       

- dev UAS: 75.44
Current UAS scores = 0.7543593631539045
New best dev UAS! Saving model.



In [113]:
numericalized_sentences = parser.numericalize(test_sentences)

UAS, dependencies = parser.parse(numericalized_sentences)

1081it [00:00, 995836.29it/s]         


In [114]:
#dependencies

In [117]:
for row in range(len(dependencies)):
    word_obj_list = []
    arcs_obj_list = []
    
    for col in range(len(dependencies[row])):
        print(dependencies[row][col])
        left, right = dependencies[row][col] #Tuple Pair
        word = final_sentences[row][col]
        word_obj = {"text": "It", "tag": "DT"}
    
    break

(6, 5)
(6, 4)
(6, 3)
(6, 2)
(6, 1)
(6, 7)
(6, 8)
(0, 6)


In [119]:
ex = {
    "words": [
        {"text": "It", "tag": "DT"},
        {"text": "was", "tag": "VBZ"},
        {"text": "a", "tag": "DT"},
        {"text": "pen", "tag": "NN"}
    ],
    "arcs": [
        {"start": 0, "end": 1, "label": "", "dir": "left"},
        {"start": 2, "end": 3, "label": "", "dir": "left"},
        {"start": 1, "end": 3, "label": "", "dir": "right"}
    ]
}
displacy.render(ex, style="dep", manual=True)